In [2]:
from GENIE3 import *

In [3]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import SAUCIE
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
from tqdm import tqdm

sys.path.append(os.getcwd())

In [4]:
sys.path

['/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3',
 '/opt/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python311.zip',
 '/opt/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11',
 '/opt/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload',
 '',
 '/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/venv/lib/python3.11/site-packages',
 '/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3']

In [5]:
y = np.transpose(np.load('../imputations/DS3/DS6_45.npy'))
y_test = np.transpose(np.load('../SERGIO/imputation_data/DS6_45.npy'))

In [6]:
x = np.transpose(np.load('../SERGIO/imputation_data/DS6_clean.npy'))

In [6]:
y_hat_saucie = np.load('../SAUCIE/yhat_saucie.npy')

In [7]:
y_magic_2 = np.load('../imputations/DS3/yhat_MAGIC_t_2.npy')
y_magic_7 = np.load('../imputations/DS3/yhat_MAGIC_t_7.npy')
y_magic_default = np.load('../imputations/DS3/yhat_MAGIC_t_auto.npy')

In [8]:
target_file = '../SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/Interaction_cID_6.txt'

In [9]:
magic_df = pd.DataFrame(y_magic_2)

In [89]:
file = open('../SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/Interaction_cID_6.txt', 'r')
lines = file.readlines()
regulators = []
for row in lines:
    row = row.split(',')
    num_regs_row = int(float(row[1]))
    if num_regs_row != 0:
        for i in range(2, 2 + num_regs_row):
            regulators.append(str(int(float(row[i]))))
file.close()
regulators = list(set(regulators))

In [90]:
master_regulators = pd.read_table('../SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/Regs_cID_6.txt', header=None, sep=',')
master_regulators = master_regulators[0].astype(int).astype(str).tolist()

In [91]:
other_regs = pd.read_csv('../SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/gt_GRN.csv', header=None)
other_regs = list(set(other_regs[0].values.astype(str).tolist()))
hmm = [x for x in other_regs if x not in master_regulators]
print(len(regulators), len(other_regs), len(master_regulators), len(true_regs), len(hmm))

127 127 67 60 60


In [92]:
true_regs = [x for x in regulators if x not in master_regulators]

In [93]:
print(len(regulators), len(master_regulators), len(true_regs))

127 67 60


In [100]:
#VIM = GENIE3(y_magic_2,nthreads=12, ntrees=100, regulators='all', gene_names=[str(s) for s in range(y.shape[1])])
#VIM_1 = GENIE3(y_magic_2,nthreads=12, ntrees=100, regulators=regulators, gene_names=[str(s) for s in range(y_magic_2.shape[1])])
#VIM_2 = GENIE3(y_magic_2,nthreads=12, ntrees=100, regulators=master_regulators, gene_names=[str(s) for s in range(y.shape[1])])
#VIM_3 = GENIE3(y_magic_2,nthreads=12, ntrees=100, regulators=true_regs, gene_names=[str(s) for s in range(y.shape[1])])

Tree method: RF
K: sqrt
Number of trees: 100


running jobs on 12 threads
Elapsed time: 491.77 seconds


In [101]:
from sklearn.metrics import roc_auc_score
for i in range(4):
    v = VIM
    if i == 1:
        print("all regulators")
        v = VIM_1
    if i == 2:
        print("master regulators")
        v = VIM_2
    if i == 3:
        print("'true' regulators")
        v = VIM_3
    if i == 0:
        print("baseline")
    gt = np.zeros_like(v)
    f = open(target_file,'r')
    Lines = f.readlines()
    f.close()
    for j in range(len(Lines)):
        line = Lines[j]
        line_list = line.split(',')
        target_index = int(float(line_list[0]))
        num_regs = int(float(line_list[1]))
        for i in range(num_regs):
            try:
                reg_index = int(float(line_list[i+2]))
                gt[reg_index,target_index] = 1 
            except:
                continue
    print(i, roc_auc_score(gt.flatten(),v.flatten()))  

baseline


2 0.8151714998867207
all regulators
2 0.9478909403744877
master regulators
2 0.7185733423314502
'true' regulators
2 0.7475585877808458


Saucie imputed data performance

0.7097546768955613

In [17]:
y_hat_scscope = np.load('../scScope/yhat_scScope.npy')

In [18]:
VIM = GENIE3(y_hat_scscope,nthreads=12,ntrees=100)

Tree method: RF
K: sqrt
Number of trees: 100


running jobs on 12 threads


/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output

KeyboardInterrupt: 

Process SpawnPoolWorker-70:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.2_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Cellar/python@3.11/3.11.2_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/Cellar/python@3.11/3.11.2_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.2_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
           ^^^^^^^^^^^^^^^^
  File "/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py", line 297, in wr_GENIE3_single
  

scScope imputed data performance

In [19]:
roc_auc_score(gt.flatten(),VIM.flatten())

0.46970084150209535

In [20]:
VIM = GENIE3(x,nthreads=12,ntrees=100)

Tree method: RF
K: sqrt
Number of trees: 100


running jobs on 12 threads


/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output

KeyboardInterrupt: 

Clean data performance

In [ ]:
roc_auc_score(gt.flatten(),VIM.flatten())

In [ ]:
VIM = GENIE3(y,nthreads=12,ntrees=100)

Noisy data performance

In [ ]:
roc_auc_score(gt.flatten(),VIM.flatten())

## DeepImpute

In [21]:
y_hat_deepimp = np.load('../deepimpute/yhat_deepimp.npy')

In [ ]:
VIM = GENIE3(y_hat_deepimp,nthreads=12,ntrees=100)

Tree method: RF
K: sqrt
Number of trees: 100


running jobs on 12 threads


/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output)
/Users/joshuaweiner/Desktop/Folders/Projects/zero_imputation/GENIE3/GENIE3.py:309: RuntimeWarning: invalid value encountered in divide
  output = output / std(output

In [ ]:
roc_auc_score(gt.flatten(),VIM.flatten())

## MAGIC

In [12]:
y_hat_magic = np.load('../MAGIC/yhat_MAGIC.npy')

In [ ]:
VIM = GENIE3(y_hat_magic, nthreads=12, ntrees=100)

In [ ]:
roc_auc_score(gt.flatten(), VIM.flatten())